# load biocaster dataset

In [24]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaser2text
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
biocaser_data_df = biocaser2text(data_file)
biocaser_data_df["source"] = "Biocaster"
biocaser_data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,1,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,1,Biocaster


# load promed_extended dataset

In [25]:
## this should load a dataframe, which col1 being the documents and col2 being the labels.
from beta_nlp.utils.data_util import get_raw_extended_promed_df
data_df_extended = get_raw_extended_promed_df()
data_df_extended.name = "extended_promed"
data_df_extended.labels = data_df_extended.labels.apply(lambda x: 1 if x==2 else x)

found 3430 files
Remain 3377 files after filter_out with alerting
found files: 3377
found 3430 files
Remain 53 files after filter_in with alerting
found files: 53
found 3862 files
found files: 3862


In [26]:
data_df_extended["source"] = "promed_extended"
data_df_extended.head()

,docs,labels,source
0,"\n \n \n \n AVIAN INFLUENZA, HUMAN (109) - IND...",1,promed_extended
1,Health officials in Hong Kong say that...,1,promed_extended
2,<h> Coronavirus Detected In Patient Quarantine...,1,promed_extended
3,\n \n EQUINE INFLUENZA - CHINA\n \n **********...,1,promed_extended
4,"\n \n \n \n SALMONELLOSIS SENFTENBERG, BASIL -...",1,promed_extended


In [27]:
data_df_extended.labels.value_counts(),len(data_df_extended.index)

(0    3862
 1    3430
 Name: labels, dtype: int64,
 7292)

In [28]:
data_df_26k = data_df_extended.append(biocaser_data_df)

In [29]:
from os import listdir
from os.path import isfile, join
import codecs
import pandas as pd

data_path = "/home/zm324/workspace/doc_cls/datasets/bbc/"
topics  = listdir(data_path)
docfiles = []
for topic_path in topics:
    docfiles += [join(data_path,topic_path,f) for f in listdir(join(data_path,topic_path)) if isfile(join(data_path,topic_path, f))]
print(f" Find {len(docfiles)} docs")
docs = []
for doc_file in docfiles:
    with codecs.open(doc_file, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()
        doc_string = " \n ".join(lines)
    docs.append(doc_string)
print(f" Read {len(docs)} docs")

 Find 2225 docs
 Read 2225 docs


In [30]:
import pandas as pd
bbc_df = pd.DataFrame({"docs":docs,"labels":[0]*len(docs),"source":["bbc"]*len(docs)})
bbc_df.head()

,docs,labels,source
0,US in EU tariff chaos trade row\n \n \n \n The...,0,bbc
1,Takeover rumour lifts Exel shares\n \n \n \n S...,0,bbc
2,Bush budget seeks deep cutbacks\n \n \n \n Pre...,0,bbc
3,Metlife buys up Citigroup insurer\n \n \n \n U...,0,bbc
4,German economy rebounds\n \n \n \n Germany's e...,0,bbc


In [31]:
bbc_df.labels.value_counts()

0    2225
Name: labels, dtype: int64

In [32]:
data_df_26k = data_df_26k.append(bbc_df)

In [33]:
data_df_26k.labels.value_counts()

0    6716
1    3804
Name: labels, dtype: int64

In [34]:
data_path = "/home/zm324/workspace/doc_cls/datasets/Reuters21578-Apte-90Cat/training/"
topics  = listdir(data_path)
docs = []
docfiles = []
for topic_path in topics:
    docfiles += [join(data_path,topic_path,f) for f in listdir(join(data_path,topic_path)) if isfile(join(data_path,topic_path, f))]
print(f" Find {len(docfiles)} docs")

data_path = "/home/zm324/workspace/doc_cls/datasets/Reuters21578-Apte-90Cat/test/"
topics  = listdir(data_path)
for topic_path in topics:
    docfiles += [join(data_path,topic_path,f) for f in listdir(join(data_path,topic_path)) if isfile(join(data_path,topic_path, f))]
print(f" Find {len(docfiles)} docs")

for doc_file in docfiles:
    with codecs.open(doc_file, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()
        doc_string = " \n ".join(lines)
    if len(doc_string)>10:
        docs.append(doc_string)
print(f" Read {len(docs)} docs")

 Find 11413 docs
 Find 15437 docs
 Read 15437 docs


In [35]:
import pandas as pd
reuters_df = pd.DataFrame({"docs":docs,"labels":[0]*len(docs),"source":["Reuters"]*len(docs)})
reuters_df.head()

,docs,labels,source
0,\n \n \r\n \n PANAMA SUGAR VESSEL SAFELY DOCKE...,0,Reuters
1,\n \n \r\n \n MOSCOW SUPPORTS FREE GULF NAVIGA...,0,Reuters
2,\n \n \r\n \n JANUARY CRUDE OIL MOVEMENTS FALL...,0,Reuters
3,\n \n \r\n \n LONDON METAL FREIGHTS\r\n \n \n ...,0,Reuters
4,\n \n \r\n \n GRAIN SHIPS WAITING AT NEW ORLEA...,0,Reuters


In [36]:
data_df_26k=data_df_26k.append(reuters_df)

In [37]:
data_df_26k.labels.value_counts(), len(data_df_26k.index)

(0    22153
 1     3804
 Name: labels, dtype: int64,
 25957)

## Remove Nan docs

In [38]:
len(data_df_26k.index)

25957

In [56]:
data_df_26k = data_df_26k[data_df_26k["docs"]!=""]
len(data_df_26k.index)

25956

In [57]:
import numpy as np
for idx,row in data_df_26k.iterrows():
    if row["docs"]=="":
        print(row)

In [58]:
import random
import numpy as np
seed = 2020
random.seed(seed)
np.random.seed(seed)

from sklearn.utils import shuffle
data_df_26k_shuffle = shuffle(data_df_26k).reset_index()
data_df_26k_shuffle

,index,docs,labels,source
0,32,India's Deccan gets more planes\n \n \n \n Air...,0,bbc
1,10431,\n \n DECLINES COMMENT ON HONGKONG LAND REPOR...,0,Reuters
2,1676,Letter to XPO staff reveals extra safety measu...,1,promed_extended
3,13747,\n \n \r\n \n KEYCORP <KEY> 3RD QTR NET\r\n \n...,0,Reuters
4,4435,\n \n \r\n \n API SAYS DISTILLATE STOCKS OFF 2...,0,Reuters
...,...,...,...,...
25951,8004,\n \n \r\n \n NEW RUBBER PACT ADOPTED AT GENEV...,0,Reuters
25952,1452,\n \n \r\n \n ABU DHABI TO REOPEN GULF OILFIEL...,0,Reuters
25953,4447,\n \n \r\n \n DRAWDOWN SEEN IN U.S. DISTILLATE...,0,Reuters
25954,200,\n HANTAVIRUS UPDATE 2006 - AMERICAS (01): CO...,1,Biocaster


In [60]:
data_df_26k_shuffle["flag"] = None
data_df_26k_shuffle.head()

,index,docs,labels,source,flag
0,32,India's Deccan gets more planes\n \n \n \n Air...,0,bbc,None
1,10431,\n \n DECLINES COMMENT ON HONGKONG LAND REPOR...,0,Reuters,None
2,1676,Letter to XPO staff reveals extra safety measu...,1,promed_extended,None
3,13747,\n \n \r\n \n KEYCORP <KEY> 3RD QTR NET\r\n \n...,0,Reuters,None
4,4435,\n \n \r\n \n API SAYS DISTILLATE STOCKS OFF 2...,0,Reuters,None


In [61]:
train_index = int(len(data_df_26k_shuffle.index)*0.8)
valid_index = train_index + round(len(data_df_26k_shuffle.index)*0.1)
data_df_26k_shuffle.iloc[:train_index]["flag"]="train"
data_df_26k_shuffle.iloc[train_index:valid_index]["flag"]="valid"
data_df_26k_shuffle.iloc[valid_index:]["flag"]="test"

<ipython-input-61-9b3f04c7d04b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_26k_shuffle.iloc[:train_index]["flag"]="train"
<ipython-input-61-9b3f04c7d04b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_26k_shuffle.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-61-9b3f04c7d04b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [62]:
data_df_26k_shuffle.flag.value_counts()

train    20764
valid     2596
test      2596
Name: flag, dtype: int64

In [63]:
data_df_26k_shuffle.head()

,index,docs,labels,source,flag
0,32,India's Deccan gets more planes\n \n \n \n Air...,0,bbc,train
1,10431,\n \n DECLINES COMMENT ON HONGKONG LAND REPOR...,0,Reuters,train
2,1676,Letter to XPO staff reveals extra safety measu...,1,promed_extended,train
3,13747,\n \n \r\n \n KEYCORP <KEY> 3RD QTR NET\r\n \n...,0,Reuters,train
4,4435,\n \n \r\n \n API SAYS DISTILLATE STOCKS OFF 2...,0,Reuters,train


In [64]:
data_df_26k_shuffle.to_csv("../datasets/biocls_26k.csv")

In [72]:
data_df_26k_shuffle[data_df_26k_shuffle["flag"]=="test"]["labels"].value_counts()

0    2212
1     384
Name: labels, dtype: int64

In [71]:
data_df_26k_shuffle[data_df_26k_shuffle["flag"]=="train"]["labels"].value_counts()

0    17715
1     3049
Name: labels, dtype: int64

In [70]:
data_df_26k_shuffle[data_df_26k_shuffle["flag"]=="valid"]["labels"].value_counts()

0    2225
1     371
Name: labels, dtype: int64

In [69]:
len(data_df_26k_shuffle.index)

25956

In [140]:
print("|length|count|percent|")
n_lengths = [50,256,512,1024,2048,4096]
for length in n_lengths:
    count = 0
    for idx,row in data_df_26k_shuffle.iterrows():
        if len(row["docs"].split(" "))<=length:
            count+=1
    print("|{:d}|{:d}|{:.4f}|".format(length,count,count/len(data_df_26k_shuffle.index)))

|length|count|percent|
|50|1436|0.0553|
|256|14973|0.5768|
|512|21087|0.8124|
|1024|24433|0.9413|
|2048|25513|0.9829|
|4096|25825|0.9949|


In [141]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
print("|length|count|percent|")
n_lengths = [50,256,512,1024,2048,4096]
for length in n_lengths:
    count = 0
    for idx,row in data_df_26k_shuffle.iterrows():
        ids = tokenizer.tokenize(row["docs"])
        if len(ids)<=length:
            count+=1
    print("|{:d}|{:d}|{:.4f}|".format(length,count,count/len(data_df_26k_shuffle.index)))

|length|count|percent|
|50|994|0.0383|
|256|13021|0.5016|
|512|19742|0.7606|
|1024|23898|0.9207|
|2048|25355|0.9768|
|4096|25799|0.9939|


In [142]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
print("|length|count|percent|")
n_lengths = [50,256,512,1024,2048,4096]
for length in n_lengths:
    count = 0
    for idx,row in data_df_26k_shuffle.iterrows():
        ids = tokenizer.tokenize(row["docs"])
        if len(ids)<=length:
            count+=1
    print("|{:d}|{:d}|{:.4f}|".format(length,count,count/len(data_df_26k_shuffle.index)))

|length|count|percent|
|50|1908|0.0735|
|256|13797|0.5315|
|512|20146|0.7761|
|1024|24034|0.9259|
|2048|25401|0.9786|
|4096|25834|0.9953|
